# Information Retrieval and Web Search
<p>
Course Project - Clustering documents to compress inverted index<br>
Giovanni Costa - 880892
</p>

In [ ]:
import numpy as np
import multiprocessing
from sklearn.cluster import MiniBatchKMeans
from sklearn.mixture import GaussianMixture

from sklearn.decomposition import TruncatedSVD
from sklearn.datasets import fetch_rcv1
from utils import random_search
from Indexer import Indexer

input_path="input/"
output_path="output/"
CORE_NUM=multiprocessing.cpu_count()
np.random.seed(42)

## Data parsing and TF-IDF representation

In [ ]:
sparse_docs, _=fetch_rcv1(return_X_y=True)
sparse_docs=sparse_docs.astype("float32")
print(sparse_docs.shape)

In [ ]:
indexer=Indexer()
std_inverted_index=indexer.get_dict_from_csr_matrix(sparse_docs)

### MiniBatch K-Means Method

In [ ]:
n_iter=15
params_k_means={"n_clusters": [i for i in range(2, 104)]}
k_means_obj=MiniBatchKMeans(batch_size=256*CORE_NUM, n_init="auto") #For faster computations, you can set the batch_size greater than 256 * number of cores to enable parallelism on all cores

In [ ]:
best_k_means, best_k_means_remapping=random_search(k_means_obj, sparse_docs, std_inverted_index, params_k_means, n_iter)

### Gaussian Mixture method

In [ ]:
n_iter=15
params_mixture={"n_components": [i for i in range(2, 104)]}
mixture_obj=GaussianMixture(covariance_type="diag", init_params='k-means++')

In [ ]:
best_mixture, best_mixture_remapping=random_search(mixture_obj, sparse_docs, std_inverted_index, params_mixture, n_iter)